<a href="https://colab.research.google.com/github/Abelbrown/Iceme-github-pages/blob/master/exploration_donnees_H%26M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Propos préliminaires

**Objectif**: prédire les achats d'un client durant les 7 jours à compter de la dernière date du training data". Les prédictions doivent être faites pour tous les customers id listés dans le sample submission file. 

**Nombre de fichiers**: 4 fichiers csv (articles, customers, sample_submission et transactions_train) et un dossier images 

**Commentaires particuliers**: 

1.   Tous les articles n'ont pas forcément une image associée
2.   Les lignes dupliquées dans le fichier 'transactions_train' correspondent à des achats multiples pour le même item
3.   Tous les clients qui ont fait des achats durant la période test (7 jours) sont pris en compte dans la notation, même si pas d'achat durant la période de training data. Les clients qui n'auront pas acheté durant ces 7 jours seront exclus du scoring
4.   La test week est la même pour tous les clients *(Réponse du host)*
5.   Les valeurs (1,2) ds la var sales_channel_id du fichier transactions signifie "achat en ligne", "achat hors ligne" *(Réponse du host)*
6.   Possible de prédire des items déjà achetés par le client *(Réponse du host)*
7.   Les prix des articles sont compris entre 0 et 0.59 -> volontairement, le host n'a pas fourni une valeur en monnaie par unité afin de ne pas dévoiler la valeur du bien *(Réponse du host)*
8.   Il peut y avoir achat de mêmes articles à différents prix en raison d'un discount (buy 1 get 50% on 2nd) *(source: forum)*
9.   Un même article peut être acheté le même jour online et offline (cf dans trans -> id 6942 et 6943) -> peut être un membre de la famille qui achète le même item *(source: forum)*
10.  FN -> newsletter : si reçoit la newsletter: donc 1 



### Téléchargement des fichers csv

In [1]:
import pandas as pd
df_sample = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')
df_trans = pd.read_csv('/content/drive/MyDrive/transactions_train.csv')
df_art = pd.read_csv('/content/drive/MyDrive/articles.csv')
df_cust = pd.read_csv('/content/drive/MyDrive/customers.csv')

## Aperçu des fichiers csv

Fichier 'Sample'

In [2]:
print(df_sample.shape)
df_sample.head(10)

(1371980, 2)


,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...
5,000064249685c11552da43ef22a5030f35a147f723d5b0...,0706016001 0706016002 0372860001 0610776002 07...
6,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,0706016001 0706016002 0372860001 0610776002 07...
7,00007d2de826758b65a93dd24ce629ed66842531df6699...,0706016001 0706016002 0372860001 0610776002 07...
8,00007e8d4e54114b5b2a9b51586325a8d0fa74ea23ef77...,0706016001 0706016002 0372860001 0610776002 07...
9,00008469a21b50b3d147c97135e25b4201a8c58997f787...,0706016001 0706016002 0372860001 0610776002 07...


Fichier "Transactions"

In [3]:
print(df_trans.shape)
df_trans.head(10)
#price is not clear
# 1 & 2 of sales_channel_id means online / offline channels

(31788324, 5)


,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
5,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687001,0.016932,2
6,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221001,0.020322,2
7,2018-09-20,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,688873012,0.030492,1
8,2018-09-20,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,501323011,0.053373,1
9,2018-09-20,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,598859003,0.045746,2


In [4]:
# Exemple d'achat même article sur deux channels différentes
#df_trans[df_trans['customer_id']=='25952508d3a9fd13159404c54d7b9d1c656fdbdbb5d2e1f61b04fdb8c30432ac']
df_trans.loc[[6942, 6943]]

,t_dat,customer_id,article_id,price,sales_channel_id
6942,2018-09-20,25952508d3a9fd13159404c54d7b9d1c656fdbdbb5d2e1...,665095002,0.025407,2
6943,2018-09-20,25952508d3a9fd13159404c54d7b9d1c656fdbdbb5d2e1...,665095002,0.025407,1


Fichier "Articles"

In [5]:
print(df_art.shape)
df_art.head(10)

(105542, 25)


,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
5,110065011,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,12,Light Beige,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
6,111565001,111565,20 den 1p Stockings,304,Underwear Tights,Socks & Tights,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo..."
7,111565003,111565,20 den 1p Stockings,302,Socks,Socks & Tights,1010016,Solid,13,Beige,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo..."
8,111586001,111586,Shape Up 30 den 1p Tights,273,Leggings/Tights,Garment Lower body,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,Tights with built-in support to lift the botto...
9,111593001,111593,Support 40 den 1p Tights,304,Underwear Tights,Socks & Tights,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny tights that shape the tummy, thighs..."


Fichier 'Customer'

In [6]:
print(df_cust.shape)
df_cust.head(10)

(1371980, 7)


,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...
5,000064249685c11552da43ef22a5030f35a147f723d5b0...,NaN,NaN,NaN,NaN,NaN,2c29ae653a9282cce4151bd87643c907644e09541abc28...
6,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,NaN,NaN,ACTIVE,NONE,20.0,fe7b8e2b3fafb89ca90db17ffeeae0fd29b795d803f749...
7,00007d2de826758b65a93dd24ce629ed66842531df6699...,1.0,1.0,ACTIVE,Regularly,32.0,8d6f45050876d059c830a0fe63f1a4c022de279bb68ce3...
8,00007e8d4e54114b5b2a9b51586325a8d0fa74ea23ef77...,NaN,NaN,ACTIVE,NONE,20.0,2c29ae653a9282cce4151bd87643c907644e09541abc28...
9,00008469a21b50b3d147c97135e25b4201a8c58997f787...,NaN,NaN,ACTIVE,NONE,20.0,2c29ae653a9282cce4151bd87643c907644e09541abc28...


## Affichage des principales infos et stats sur chaque fichier csv

Fichier 'Sample'


In [7]:
print(df_sample.info())
df_sample.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 2 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   customer_id  1371980 non-null  object
 1   prediction   1371980 non-null  object
dtypes: object(2)
memory usage: 20.9+ MB
None


,customer_id,prediction
count,1371980,1371980
unique,1371980,1
top,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
freq,1,1371980


Fichier 'Transactions'

In [8]:
print(df_trans.info())
df_trans.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ GB
None


,article_id,price,sales_channel_id
count,3.178832e+07,3.178832e+07,3.178832e+07
mean,6.962272e+08,2.782927e-02,1.704028e+00
std,1.334480e+08,1.918113e-02,4.564786e-01
min,1.087750e+08,1.694915e-05,1.000000e+00
25%,6.328030e+08,1.581356e-02,1.000000e+00
50%,7.145820e+08,2.540678e-02,2.000000e+00
75%,7.865240e+08,3.388136e-02,2.000000e+00
max,9.562170e+08,5.915254e-01,2.000000e+00


Fichier 'Article'

In [9]:
print(df_art.info())
df_art.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

,article_id,product_code,product_type_no,graphical_appearance_no,colour_group_code,perceived_colour_value_id,perceived_colour_master_id,department_no,index_group_no,section_no,garment_group_no
count,1.055420e+05,105542.000000,105542.000000,1.055420e+05,105542.000000,105542.000000,105542.000000,105542.000000,105542.000000,105542.000000,105542.000000
mean,6.984246e+08,698424.563378,234.861875,1.009515e+06,32.233822,3.206183,7.807972,4532.777833,3.171534,42.664219,1010.438290
std,1.284624e+08,128462.384432,75.049308,2.241359e+04,28.086154,1.563839,5.376727,2712.692011,4.353234,23.260105,6.731023
min,1.087750e+08,108775.000000,-1.000000,-1.000000e+00,-1.000000,-1.000000,-1.000000,1201.000000,1.000000,2.000000,1001.000000
25%,6.169925e+08,616992.500000,252.000000,1.010008e+06,9.000000,2.000000,4.000000,1676.000000,1.000000,20.000000,1005.000000
50%,7.022130e+08,702213.000000,259.000000,1.010016e+06,14.000000,4.000000,5.000000,4222.000000,2.000000,46.000000,1009.000000
75%,7.967030e+08,796703.000000,272.000000,1.010016e+06,52.000000,4.000000,11.000000,7389.000000,4.000000,61.000000,1017.000000
max,9.594610e+08,959461.000000,762.000000,1.010029e+06,93.000000,7.000000,20.000000,9989.000000,26.000000,97.000000,1025.000000


Fichier 'Customer'

In [10]:
print(df_cust.info())
df_cust.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB
None


,FN,Active,age
count,476930.0,464404.0,1.356119e+06
mean,1.0,1.0,3.638696e+01
std,0.0,0.0,1.431363e+01
min,1.0,1.0,1.600000e+01
25%,1.0,1.0,2.400000e+01
50%,1.0,1.0,3.200000e+01
75%,1.0,1.0,4.900000e+01
max,1.0,1.0,9.900000e+01


## Ciblage des données manquantes

In [11]:
df_sample.isna().sum()

customer_id    0
prediction     0
dtype: int64

In [12]:
df_trans.isna().sum()

t_dat               0
customer_id         0
article_id          0
price               0
sales_channel_id    0
dtype: int64

In [13]:
df_art.isna().sum()
#416 données manquantes de la variable detail_desc

article_id                        0
product_code                      0
prod_name                         0
product_type_no                   0
product_type_name                 0
product_group_name                0
graphical_appearance_no           0
graphical_appearance_name         0
colour_group_code                 0
colour_group_name                 0
perceived_colour_value_id         0
perceived_colour_value_name       0
perceived_colour_master_id        0
perceived_colour_master_name      0
department_no                     0
department_name                   0
index_code                        0
index_name                        0
index_group_no                    0
index_group_name                  0
section_no                        0
section_name                      0
garment_group_no                  0
garment_group_name                0
detail_desc                     416
dtype: int64

In [14]:
df_cust.isna().sum()
# A lot of data missing in the customer file 

customer_id                    0
FN                        895050
Active                    907576
club_member_status          6062
fashion_news_frequency     16009
age                        15861
postal_code                    0
dtype: int64

# Fixation de la BDD

### Nettoyage du jeu de données

In [15]:
# Étape 1: repérage de doublons

print("Le nombre de doublons présents dans le fichier sample est de :", df_sample.duplicated().sum())
print("Le nombre de doublons présents dans le fichier transactions est de :", df_trans.duplicated().sum())
print("Le nombre de doublons présents dans le fichier articles est de :", df_art.duplicated().sum())
print("Le nombre de doublons présents dans le fichier customers est de :", df_cust.duplicated().sum())

Le nombre de doublons présents dans le fichier sample est de : 0
Le nombre de doublons présents dans le fichier transactions est de : 2974905
Le nombre de doublons présents dans le fichier articles est de : 0
Le nombre de doublons présents dans le fichier customers est de : 0


In [16]:
# Étape 2: suppression des doublons ?
# df_trans = df_trans.drop_duplicates()  ## not smart to do that bc duplicated lines in transactions refer to multiple buying of same items


In [17]:
# Étape 3: Gestion des NaN
 ## fichier 'article' 
   ### Detection des lignes avec valeurs manquantes
df_art[df_art.isna().any(axis =1)]
   ### Proportion du nombre de lignes avec NaN 
print("La proportion de lignes du fichier article avec une valeur manquante est de:", '{:.1%}'.format((df_art['detail_desc'].isnull().sum() / df_art.shape[0])))
   ### Choix de suppression de ces lignes et création d'un nouveau df 
df_art = df_art.dropna(axis = 0, how='all', subset=['detail_desc'])

 ## fichier 'customer'
   ### Colonne FN
     #### Inspection colonne  
df_cust['FN'].head(30)
     #### Remplacement des NaN par 0
df_cust['FN'] = df_cust['FN'].fillna(0)

   ### Colonne Active
     #### Inspection colonne 
df_cust['Active'].head(30)
     #### Remplacement des NaN par 0
df_cust['Active'] = df_cust['Active'].fillna(0)

   ### Colonne Club member status
     #### Inspection colonne 
df_cust['club_member_status'].head(30)
     #### Remplacement des NaN par NON ACTIVE
df_cust['club_member_status'] = df_cust['club_member_status'].fillna('NON ACTIVE')
df_cust[df_cust['club_member_status']=='NON ACTIVE'] 

   ### Colonne Fashion News Frequency
     #### Inspection colonne 
df_cust['fashion_news_frequency'].head(30)     
df_cust[df_cust['fashion_news_frequency'].isna()]  # Pour voir le lien entre cette colonne (avec modalités NaN) et les autres colonnes 
     #### Option 1: Remplacement NaN par None
df_cust['fashion_news_frequency'] = df_cust['fashion_news_frequency'].fillna('NONE')
     #### Option 2: Suppression des lignes qui ont des valeurs manquantes dans age et fashions_news_freq
#df_cust[(df_cust['fashion_news_frequency'].isna()) &(df_cust['age'].isna())]  --> detecter les lignes qui ont val manquantes dans 2 colonnes
#df_cust = df_cust.dropna(axis = 0, how='all', subset=['fashion_news_frequency', 'age']) --> suppression des lignes
     #### Option 3 (suggestion de Jean) -> garder les lignes avec NaN car explication logique

   ### Colonne age
     #### Inspection de la colonne 
df_cust['age'].head(30)
df_cust[df_cust['age'].isna()]  # Pour voir le lien entre cette colonne (avec modalités NaN) et les autres colonnes
     #### Option 1: Remplacement NaN par None
df_cust['age'] = df_cust['age'].fillna('NONE')
     #### Option 2: Remplacement par la moyenne
#df_cust['age'] = df_cust['age'].fillna(df_cust['age'].mean())
   
  ### Vérification qu'absence de NaN 
df_cust.isna().sum()

La proportion de lignes du fichier article avec une valeur manquante est de: 0.4%


customer_id               0
FN                        0
Active                    0
club_member_status        0
fashion_news_frequency    0
age                       0
postal_code               0
dtype: int64

In [18]:
 # Étape 4: Vérification qu'absence de NaN dans tous nos fichiers 
  ## fichier 'article'
print(df_art.isna().sum()), '\n'

  ## fichier 'customer'

print(df_cust.isna().sum())

article_id                      0
product_code                    0
prod_name                       0
product_type_no                 0
product_type_name               0
product_group_name              0
graphical_appearance_no         0
graphical_appearance_name       0
colour_group_code               0
colour_group_name               0
perceived_colour_value_id       0
perceived_colour_value_name     0
perceived_colour_master_id      0
perceived_colour_master_name    0
department_no                   0
department_name                 0
index_code                      0
index_name                      0
index_group_no                  0
index_group_name                0
section_no                      0
section_name                    0
garment_group_no                0
garment_group_name              0
detail_desc                     0
dtype: int64
customer_id               0
FN                        0
Active                    0
club_member_status        0
fashion_news_frequency   

### Jointures


In [19]:
# Fusionner fichier 


In [22]:
df_trans.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [23]:
df_art.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
